# Train YOLO Pose Model (Bee Dataset)

This notebook trains a YOLO pose estimation model using data in `./labeled-data` (YOLO v1.0 pose labels).

## 1) Install/Import Dependencies

In [11]:
# If needed, uncomment to install Ultralytics
# %pip install -q ultralytics pyyaml

import os
from pathlib import Path

from ultralytics import YOLO

## 2) Configure Paths

In [12]:
ROOT = Path.cwd()
DATA_DIR = ROOT / "labeled-data"
IMAGES_DIR = DATA_DIR / "images"
LABELS_DIR = DATA_DIR / "labels"
DATA_YAML = DATA_DIR / "data.yaml"

assert DATA_DIR.exists(), f"Missing dataset folder: {DATA_DIR}"
assert IMAGES_DIR.exists(), f"Missing images folder: {IMAGES_DIR}"
assert LABELS_DIR.exists(), f"Missing labels folder: {LABELS_DIR}"
assert DATA_YAML.exists(), f"Missing YAML file: {DATA_YAML}"

print(f"Workspace root: {ROOT}")
print(f"Dataset folder: {DATA_DIR}")

Workspace root: c:\Users\bee-ops\code\Choice-assay
Dataset folder: c:\Users\bee-ops\code\Choice-assay\labeled-data


## 3a) Remove images without labels

In [13]:
# Remove any images that don't have corresponding labels
for img_file in IMAGES_DIR.glob("*.png"):
    label_file = LABELS_DIR / img_file.with_suffix(".txt").name
    if not label_file.exists():
        print(f"Removing image without label: {img_file}")
        img_file.unlink()

## 3b) Redo autosplit if required

In [14]:
# Autosplit the labeled-data into train, test, and validation sets
from ultralytics.data.split import autosplit


def redo_autosplit():
    # Ensure that the autosplit is run in the labeled-data directory, but return to the base directory
    # afterwards so that YOLO runs properly
    path = Path.cwd()
    if path.parts[-1] != "labeled-data":
        labeled_data_path = path / "labeled-data"
        os.chdir(labeled_data_path)
    else:
        path = path.parent
    autosplit(path=".", weights=(0.8, 0.12, 0.08))
    os.chdir(path)


redo_autosplit()

Autosplitting images from .
: 100% ━━━━━━━━━━━━ 352/352 4.4Kit/s 0.1s


## 4) Train YOLO Pose

In [ ]:
PRETRAINED_WEIGHTS = "yolov8n-pose.pt"
IMGSZ = 448

print(f"Using device training YOLO pose model: {PRETRAINED_WEIGHTS}")

model = YOLO(PRETRAINED_WEIGHTS)
results = model.train(
    data=str(DATA_YAML), epochs=300, imgsz=IMGSZ, project="runs/pose", name="bee_pose", exist_ok=True
)
results

Using device training YOLO pose model: yolov8n-pose.pt
New https://pypi.org/project/ultralytics/8.4.16 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.6  Python-3.12.10 torch-2.10.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\bee-ops\code\Choice-assay\labeled-data\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=448, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train

## 5) Validate and Export

In [ ]:
metrics = model.val(data=str(DATA_YAML))
metrics

In [ ]:
best_model = Path(model.trainer.best) if hasattr(model, "trainer") else None
print("Best model path:", best_model)

# Optional exports:
# model.export(format='onnx')
# model.export(format='torchscript')